In [8]:
data_path = '../data'
output_path = '../output'

In [9]:
import torch
import torch.nn as nn

import json
import os

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [10]:
# 读取数据
data_j = []
with open(os.path.join(data_path, 'filteredpoem2985_data.json'), encoding='utf-8') as f:
    data_j = json.load(f)
data_j[0]

{'poem_name': '青玉案·元夕',
 'author': '(辛弃疾)',
 'content': '东风夜放花千树，更吹落、星如雨。宝马雕车香满路。凤箫声动，玉壶光转，一夜鱼龙舞。蛾儿雪柳黄金缕，笑语盈盈暗香去。众里寻他千百度。蓦然回首，那人却在，灯火阑珊处。',
 'story': '这首词作于南宋淳熙元年（公元年）或淳熙二年（公元年）。当时，强敌压境，国势日衰，而南宋统治阶级却沉湎于歌舞享乐，以粉饰太平。洞察形势的":"""辛弃疾，欲补天穹，却恨无路请缨。他满腔激情、怨恨，交织成了这幅元夕求索图阴历正月十五为上元节，这日晚上称元夕，亦称元宵，元夜。我国古代有元夕观灯的风俗。玉壶：指月亮。鱼龙舞：指舞鱼灯、龙灯之类。这是一首别有寄托的词作。词人假借对一位厌恶热闹、自甘寂寞的女子的寻求，含蓄地表达了自己的高洁志向和情怀．梁启超《艺蘅馆胡词选》云：“自怜幽独，伤心人别有怀抱。”其体会是可信的。词的上片，极写元夕灯火辉煌、歌舞繁盛的热闹景象。“东风夜放花千树，更吹落、星如雨，”前一句写灯，后一句写焰火。上元之夜，满城灯火，就象一夜春风吹开了千树万树的繁花，满天的焰火明灭，又象是春风把满天星斗吹落。真是一片灯的海洋，焰火的世界，令人眼花缭乱《青玉案·元夕》是古代词作中描写“上元佳节”主题的佳作。整首词巧妙地运用了对比手法，上阕描绘了花灯璀璨、乐声悠扬的元夕盛况，犹如一幅繁华热闹的画卷。而下阕则以寻找孤高女子为中心，透过灯火阑珊的细节，展现了她的独特魅力，这样的描写方式构思精妙，语言精致，含蓄婉转，余味无穷。词人从极力渲染元宵节绚丽多彩的热闹场面入手，反衬出一个孤高淡泊、超群拔俗、不同于金翠脂粉的女性形象，寄托着作者政治失意后不愿与世俗同流合污的孤高品格。'}

In [11]:
# 把作者的括号去掉
for dic in data_j:
    dic['author'] = dic['author'][1:-1]
data_j[0]

{'poem_name': '青玉案·元夕',
 'author': '辛弃疾',
 'content': '东风夜放花千树，更吹落、星如雨。宝马雕车香满路。凤箫声动，玉壶光转，一夜鱼龙舞。蛾儿雪柳黄金缕，笑语盈盈暗香去。众里寻他千百度。蓦然回首，那人却在，灯火阑珊处。',
 'story': '这首词作于南宋淳熙元年（公元年）或淳熙二年（公元年）。当时，强敌压境，国势日衰，而南宋统治阶级却沉湎于歌舞享乐，以粉饰太平。洞察形势的":"""辛弃疾，欲补天穹，却恨无路请缨。他满腔激情、怨恨，交织成了这幅元夕求索图阴历正月十五为上元节，这日晚上称元夕，亦称元宵，元夜。我国古代有元夕观灯的风俗。玉壶：指月亮。鱼龙舞：指舞鱼灯、龙灯之类。这是一首别有寄托的词作。词人假借对一位厌恶热闹、自甘寂寞的女子的寻求，含蓄地表达了自己的高洁志向和情怀．梁启超《艺蘅馆胡词选》云：“自怜幽独，伤心人别有怀抱。”其体会是可信的。词的上片，极写元夕灯火辉煌、歌舞繁盛的热闹景象。“东风夜放花千树，更吹落、星如雨，”前一句写灯，后一句写焰火。上元之夜，满城灯火，就象一夜春风吹开了千树万树的繁花，满天的焰火明灭，又象是春风把满天星斗吹落。真是一片灯的海洋，焰火的世界，令人眼花缭乱《青玉案·元夕》是古代词作中描写“上元佳节”主题的佳作。整首词巧妙地运用了对比手法，上阕描绘了花灯璀璨、乐声悠扬的元夕盛况，犹如一幅繁华热闹的画卷。而下阕则以寻找孤高女子为中心，透过灯火阑珊的细节，展现了她的独特魅力，这样的描写方式构思精妙，语言精致，含蓄婉转，余味无穷。词人从极力渲染元宵节绚丽多彩的热闹场面入手，反衬出一个孤高淡泊、超群拔俗、不同于金翠脂粉的女性形象，寄托着作者政治失意后不愿与世俗同流合污的孤高品格。'}

# 定义模型

In [47]:
from transformers import BertTokenizer, GPT2LMHeadModel, TextGenerationPipeline
tokenizer = BertTokenizer.from_pretrained(os.path.join(output_path, "chn-gpt2-tokenizer"))
model = GPT2LMHeadModel.from_pretrained(os.path.join(output_path, "chn-gpt2-poem-untrained"))

In [14]:
torch.cuda.empty_cache()

In [15]:
import re
from torch.utils.data import Dataset, DataLoader

from torch.optim import AdamW
from tqdm import tqdm


In [16]:
batch_size = 12

In [17]:
class TextDataset(Dataset):
    def __init__(self, input_dicts, tokenizer, max_length=200):
        self.input_dicts = input_dicts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.input_dicts)
    
    import re

    def __classify_chinese_poem(self, poem):
        # 按逗号或句号拆分诗句
        lines = re.split(r'[，。]', poem)
        # 去掉空行
        lines = [line.strip() for line in lines if line.strip()]
        # 获取每行的字数
        line_lengths = [len(line) for line in lines]

        if len(lines) == 4:
            if all(length == 5 for length in line_lengths):
                return "五言绝句"
            elif all(length == 7 for length in line_lengths):
                return "七言绝句"
            else:
                return "诗"
        elif len(lines) == 8:
            if all(length == 5 for length in line_lengths):
                return "五言律诗"
            elif all(length == 7 for length in line_lengths):
                return "七言律诗"
            else:
                return "诗"
        else:
            return "诗"


    def __getitem__(self, idx):
        inp = self.input_dicts[idx]
        input_text = '根据以下故事写一首' + self.__classify_chinese_poem(inp['content']) + ': ' + inp['story']
        output_text = '[诗名]' + inp['poem_name'] + '[内容]' + inp['content']

        inputs = self.tokenizer(input_text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors="pt")
        targets = self.tokenizer(output_text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors="pt")
        
        input_ids = inputs['input_ids'].squeeze()
        attention_mask = inputs['attention_mask'].squeeze()
        labels = targets['input_ids'].squeeze()
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

dataset = TextDataset(data_j, tokenizer)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

## 定义判别模型

In [48]:
class ScoringModel(nn.Module):
    def __init__(self, hidden_dim=200, num_layers=3, num_classes=1):
        super(ScoringModel, self).__init__()

        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # LSTM层
        self.lstm = nn.LSTM(input_size=1, hidden_size=hidden_dim, num_layers=num_layers, bidirectional=True, batch_first=True)
        
        # 全连接层
        self.fc = nn.Linear(hidden_dim * 2, num_classes)

        # Sigmoid激活函数
        self.sm = nn.Sigmoid()

    def forward(self, x):
        x = x.float().to(self.fc.weight.device)
        # LSTM层的输入应该是三维张量 [batch_size, sequence_length, input_size]
        # 在这里，input_size 是 1，因为我们的输入张量的每个元素都是一个标量值
        lstm_out, _ = self.lstm(x.unsqueeze(-1))  # 添加一个维度以适应 LSTM 的输入要求
        # 取 LSTM 输出的最后一个时间步的隐藏状态作为模型的输出
        out = self.fc(lstm_out[:, -1, :])
        # 使用 Sigmoid 函数将输出限制在 [0, 1] 范围内
        out = self.sm(out)
        return out
    
scoringmodel = ScoringModel().to(device)

In [19]:
tokenizer.save_pretrained(os.path.join(output_path, "chn-gpt2-tokenizer"))
model.save_pretrained(os.path.join(output_path, "chn-gpt2-poem-untrained"))
torch.save(scoringmodel.state_dict(), os.path.join(output_path, "scoring-untrained.pt"))

### 训练打分模型
法一：先训练一个小模型，每个epoch的输出得分高于前面的输出，然后用这些数据训练打分器
法二：GAN

In [54]:
# loss
criterion = nn.BCELoss()
# 定义优化器
optimizer_G = AdamW(model.parameters(), lr=1e-4)
optimizer_D = AdamW(scoringmodel.parameters(), lr=1e-5)

# 设置训练参数
epochs = 10
model.to(device)
# torch.autograd.set_detect_anomaly(True)

generated_poems = []
fake_s = []
real_s = []
# 训练循环
for epoch in range(epochs):
    model.train()
    loop = tqdm(dataloader, leave=True)
    count = 0
    
    f_s = []
    r_s = []

    g_poems = []
    for batch in loop:
        torch.cuda.empty_cache()
        count += 1
        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        batch_size = input_ids.size(0)
        
        # 创建标签
        
        fake_poems = model(input_ids, attention_mask=attention_mask, labels=labels).logits
        fake_poems = torch.argmax(fake_poems, dim=-1)

        g_poems.append(tokenizer.decode(fake_poems[0], skip_special_tokens=True))

        # ---------------------
        #  训练判别器
        # ---------------------
        optimizer_D.zero_grad()
        
        # 真实数据的判别器损失
        scored_real = scoringmodel(labels)
        r_s.append(scored_real.cpu().detach().numpy())
        
        real_labels = torch.ones(batch_size, 1).to(device)
        d_loss_real = criterion(scored_real, real_labels)
        
        # 生成数据的判别器损失
        scored_fake = scoringmodel(fake_poems.detach())
        f_s.append(scored_real.cpu().detach().numpy())
        
        fake_labels = torch.zeros(batch_size, 1).to(device)
        d_loss_fake = criterion(scored_fake, fake_labels)
        
        # 总的判别器损失
        d_loss = d_loss_real + d_loss_fake
        if count % 8 == 0:
            d_loss.backward(retain_graph=True)
            optimizer_D.step()
        
        # ---------------------
        #  训练生成器
        # ---------------------
        optimizer_G.zero_grad()
        
        # 生成器损失，生成器希望判别器认为生成的诗是真的
        g_loss = criterion(scoringmodel(fake_poems.detach()), real_labels)
        g_loss.backward()
        optimizer_G.step()
    
        # 更新进度条
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(g_loss=g_loss.item(), d_loss=d_loss.item())

        torch.cuda.empty_cache()
        
        
    fake_s.append(f_s)
    real_s.append(r_s)
    generated_poems.append(g_poems)

Epoch 9: 100%|██████████| 249/249 [02:18<00:00,  1.80it/s, d_loss=0.0302, g_loss=4.72]


In [ ]:
# scoringmodel = scoringmodel.to(device)

# # 损失函数和优化器
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# num_epochs = 3

# # 训练模型
# for epoch in range(num_epochs):
#     model.train()
#     running_loss = 0.0
#     loop = tqdm(dataloader, leave=True)
#     for batch in loop:
#         inputs, labels = batch['input_ids'].to(device), batch['labels'].to(device)
        
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
        
#         running_loss += loss.item() * inputs.size(0)
    
#     epoch_loss = running_loss / len(dataloader.dataset)
#     print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

In [23]:
def generateOne(model, input_text, max_length=200):
    model.config.pad_token_id = model.config.eos_token_id
    model.eval()
    model = model.to(device)
    input_ids = tokenizer.encode(input_text, truncation=True, padding='max_length', max_length=max_length, return_tensors="pt").to(device)

    # 使用模型生成文本
    with torch.no_grad():
        outputs = model.generate(
            input_ids, 
            max_length=max_length+1, 
            num_beams=5, 
            early_stopping=True,
            pad_token_id=model.config.eos_token_id
        )

    # 解码生成的文本
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

In [55]:
generateOne(model, "请根据以下故事写一首五言绝句：我今天被贬离京，在路上很伤心，但是有人帮助我，我感到心里很暖")

'请 根 据 以 下 故 事 写 一 首 五 言 绝 句 ： 我 今 天 被 贬 离 京 ， 在 路 上 很 伤 心 ， 但 是 有 人 帮 助 我 ， 我 感 到 心 里 很 暖'

In [25]:
model1 = GPT2LMHeadModel.from_pretrained(os.path.join(output_path, "chn-gpt2-untrained")).to(device)

In [56]:
generateOne(model1, "你是什么人")

'你 是 什 么 人'

In [27]:
model.save_pretrained(os.path.join(output_path, "chn-gpt2-poem1"))
torch.save(scoringmodel.state_dict(), os.path.join(output_path, "scoring1.pt"))

In [57]:
generated_poems[0]

['一 柢 龙 为 维 为 留 ， 何 山 人 。 即 题 呵 。 人 ， 。 ， 意 不 一 怀 而 无 画 三 。 。 寞 。 而 我 有 时 何 风 ， 我 迹 入 其 天 无 ， ， 对 。 ， 我 时 迹 。 人 ， ， ， 。 今 不 新 心 无 。 风 ， 。 ， 。 。 。 无 一 默 ， ， 了 ， ， ， ， ， ， ， 。 ， ， 何 丝 ， 一 此 其 兮 。 我 愫 。 意 事 意 ， ， ， 此 不 一 意 ， 此 物 著 莽 。 然 一 纸 风 纸 纸 造 造 纸 一 奇 毫 之 毫 一 毫 毫 情 情 之 情 之 情 情 何 何 情 情 之 何 心 笔 造 毫 情 毫 情 何 情 毫 毫 之 心 情 毫 毫 毫 之 之 何 之 之 之 之 毫 之 毫 毫 何 风 情 何 毫 中 毫 何 毫 情 何 之 情 毫 情 之 毫 笔 何 毫 之 情 何 情 之',
 '一 本 仙 为 地 ， 详 ， 以 尾 苏 。 为 砾 吟 。 生 ， 寻 。 意 愿 德 ， 一 之 人 是 不 不 面 。 。 而 我 笑 我 之 时 之 也 。 。 是 能 ， 我 此 天 法 何 不 谓 者 而 而 生 。 不 。 一 。 之 见 我 。 ， ， 之 ， 何 河 诸 不 是 游 际 。 人 。 外 ， 况 以 前 ， 一 视 此 儿 。 不 是 如 切 意 。 是 一 个 ， 舞 ， 意 ， 恻 ， 不 与 竭 肺 输 一 谁 与 ， 。 心 。 样 当 门 路 ， ， ， ， 我 以 诗 。 风 亦 。 见 。 为 。 。 。 。 。 。 。 ， 赏 。 。 。 一 。 。 。 。 。 。 。 。 。 ， 。 说 。 我 。 。 ， 般 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 一 。 在 。 。 道 。 。 。 谁 。',
 '一 柢 龙 为 维 ， 堂 ， 以 形 斜 ， 不 三 乱 。 。 ， 不 ， 亦 不 画 ， 华 。 之 国 我 处 墨 。 是 清 行 。 以 ， 熙 ， 。 。 公 花 蛙 舞 枝 。 声 。 晴 ， 来 其 枝 为 不 姝 ， ， 的 之 中 不 他 一 时 天 玉 卦 。 声 事 子 。 曷 上 的 。 ， 今 天 才 事 不 墨 国 平 词 人 咏 性 平 。 。 。 远 。 。 ， 之 。 。 

In [53]:
fake_s[0][200]

array([[0.51629055],
       [0.51629055],
       [0.51629055],
       [0.51629055],
       [0.51629055],
       [0.51629055],
       [0.51629055],
       [0.51629055],
       [0.51629055],
       [0.51629055],
       [0.51629055],
       [0.51629055]], dtype=float32)